In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
# 定义深度可分离卷积层
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super(DepthwiseSeparableConv, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, padding=padding, groups=in_channels)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1)
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

# 定义Fire模块
class FireModule(nn.Module):
    def __init__(self, in_channels, squeeze_channels, expand_channels):
        super(FireModule, self).__init__()
        self.squeeze = nn.Conv2d(in_channels, squeeze_channels, kernel_size=1)
        self.expand1x1 = nn.Conv2d(squeeze_channels, expand_channels, kernel_size=1)
        self.expand3x3 = nn.Conv2d(squeeze_channels, expand_channels, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = self.squeeze(x)
        x = torch.relu(x)
        return torch.cat([
            torch.relu(self.expand1x1(x)),
            torch.relu(self.expand3x3(x))
        ], 1)

# 定义卷积神经网络
class DSC_CNN(nn.Module):
    def __init__(self):
        super(DSC_CNN, self).__init__()
        self.layer1 = DepthwiseSeparableConv(1, 32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fire = FireModule(32, 16, 32)
        self.concat_conv = DepthwiseSeparableConv(64, 64) # 64 because FireModule output is concatenated
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.5) # 推理时默认不会调用
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 4)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.fire(x)
        x = self.concat_conv(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.dropout(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        # 改动1：避免softmax重复计算
        x = self.softmax(x) 
        return x

In [22]:
import torch
import torch.onnx
#from tinynet import tinynet
import os

def pth_to_onnx(input, checkpoint, onnx_path, input_names=['input'], output_names=['output'], device='cpu'):
    if not onnx_path.endswith('.onnx'):
        print('Warning! The onnx model name is not correct,\
              please give a name that ends with \'.onnx\'!')
        return 0

    #model = tinynet() #导入模型
    model = DSC_CNN()
    model.load_state_dict(torch.load(checkpoint, map_location=torch.device(device))) #初始化权重
    model.eval()
    model.to(device)
    input = input.to(device)
    
    torch.onnx.export(model, input, onnx_path, verbose=True, input_names=input_names, output_names=output_names) #指定模型的输入，以及onnx的输出路径
    print("Exporting .pth model to onnx model has been successful!")

if __name__ == '__main__':
    model_name = 'model01'
    os.environ['CUDA_VISIBLE_DEVICES']='2'
    device = torch.device("cuda:2" if torch.cuda.is_available() else 'cpu')
    checkpoint = '../save/' + model_name + ".pth"
    checkpoint = '../save/' + model_name + ".pth"
    onnx_path = '../save/' + model_name +".onnx" 
    input = torch.randn(1, 1, 32, 32)
    pth_to_onnx(input, checkpoint, onnx_path, device=device)

Exported graph: graph(%input : Float(1, 1, 32, 32, strides=[1024, 1024, 32, 1], requires_grad=0, device=cpu),
      %layer1.depthwise.weight : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cpu),
      %layer1.depthwise.bias : Float(1, strides=[1], requires_grad=1, device=cpu),
      %layer1.pointwise.weight : Float(32, 1, 1, 1, strides=[1, 1, 1, 1], requires_grad=1, device=cpu),
      %layer1.pointwise.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %fire.squeeze.weight : Float(16, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cpu),
      %fire.squeeze.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %fire.expand1x1.weight : Float(32, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=1, device=cpu),
      %fire.expand1x1.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %fire.expand3x3.weight : Float(32, 16, 3, 3, strides=[144, 9, 3, 1], requires_grad=1, device=cpu),
      %fire.expand3x3.bias : Float(32, stri

In [23]:
import h5py
def read_hdf5(filename, dataset_name='dataset'):
    with h5py.File(filename, 'r') as f:
        if dataset_name in f:
            data = f[dataset_name][:]
        else:
            raise KeyError(f"数据集 '{dataset_name}' 不存在")
    return torch.tensor(data)

filename = '../data/try/data_test_small.h5'

data = read_hdf5(filename)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../data/try/data_test_small.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
!pip install onnxruntime

In [24]:
import onnxruntime as ort
import numpy as np

# 加载 ONNX 模型
onnx_path = '../save/' + model_name +".onnx"
session = ort.InferenceSession(onnx_path)

# 获取模型的输入名称和输入形状
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_type = session.get_inputs()[0].type

# 打印输入信息
print(f"Input Name: {input_name}")
print(f"Input Shape: {input_shape}")
print(f"Input Type: {input_type}")

# 创建一个模拟输入数据
# 注意：这里需要根据模型的实际输入形状和数据类型进行调整
input_data = np.random.rand(*input_shape).astype(np.float32)

# 运行推理
output = session.run(None, {input_name: input_data})

# 获取输出
# 如果模型有多个输出，可以通过 session.get_outputs() 获取所有输出名称
print("Output:", output)

Input Name: input
Input Shape: [1, 1, 32, 32]
Input Type: tensor(float)
Output: [array([[3.7362389e-07, 1.0970704e-02, 1.1392456e-06, 9.8902774e-01]],
      dtype=float32)]
